# Import modules

In [1]:
# Import standard modules
import numpy as np
import pickle
import gc
import os
import time

# Import GCMC scripts
from GCMC_data_utils import *
from GCMC_metrics import *
from GCMC_layers import *
from GCMC_model import *

# Import NN framework
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn
from torch.utils import data
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import BatchSampler, SequentialSampler
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torchvision import models

## Set directories

In [2]:
_dir = "D:\Python\Thesis\gcmc_ready_data\\"
os.chdir(_dir)

# Define parameters of the NN

In [3]:
# NN Model dim parameters
data_percent = 3 # choose the sampled dataset
pickle_in = open("gcmc_dims" + f'{data_percent}' +".pickle", "rb")
dims = pickle.load(pickle_in)

num_users = dims['Num_users']
num_items = dims['Num_items']
num_classes = dims['Num_classes']
num_side_features = dims['Num_side_features']

# NN hyper-parameters
nb = 2
emb_dim = 32
hidden = [64,32,16,8]
dropout = 0.7 # Dropout rate
num_epochs = 100
val_step = 5
test_epoch = 50
start_epoch = 0
neg_cnt = 100

lr = 0.01 # Adam param
beta1 = 0.5 # Adam param
beta2 = 0.999 # Adam param
#decay = 5e-4 # Adam param

In [4]:
dims

{'Num_users': 1479,
 'Num_items': 14427,
 'Num_classes': 2,
 'Num_features': 15906,
 'Num_side_features': 15906}

# Define directory to save models

In [5]:
model_path = "D:\Python\Thesis\gcmc_models\\"

# Set up to use GPU // CPU when necessary

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Import the prepared data sets 

In [7]:
# Read the necessary files that were created by from GCMC_create_data.ipynb

# Read the features

u_f = sparse.load_npz(file = 'u_features'+ f'{data_percent}' +'.npz').toarray() # u features
v_f = sparse.load_npz(file = 'v_features'+ f'{data_percent}' +'.npz').toarray() # v features
u_fs = sparse.load_npz(file = 'u_features_side'+ f'{data_percent}' +'.npz').toarray() # u side feat
v_fs = sparse.load_npz(file = 'v_features_side'+ f'{data_percent}' +'.npz').toarray() # v side feat

u_features = torch.from_numpy(u_f).float().to(device)
v_features = torch.from_numpy(v_f).float().to(device)
u_features_side = torch.from_numpy(u_fs).to(device)
v_features_side = torch.from_numpy(v_fs).to(device)

# Read the train, val and test splits, assumes features and split data are in same directory
rating_train = torch.load(f'{data_percent}' +'ratings_tensor_train.pkl').to(device)
rating_val = torch.load(f'{data_percent}' +'ratings_tensor_val.pkl').to(device)
rating_test = torch.load(f'{data_percent}' +'ratings_tensor_test.pkl').to(device)

# Model training

## Model definition and show # of parameters

In [8]:
# Define the Graph Auto Encoder model
model = GAE(num_users, num_items, num_classes,
            num_side_features, nb,
            u_features, v_features, u_features_side, v_features_side,
            num_users+num_items, emb_dim, hidden, dropout)

In [9]:
# Print parameters
if torch.cuda.is_available():
    model.cuda()
"""Print out the network information."""
num_params = 0
for p in model.parameters():
    num_params += p.numel()
print(model)
print("The number of parameters: {}".format(num_params))

GAE(
  (gcl1): GraphConvolution(
    (dropout): Dropout(p=0.7)
  )
  (gcl2): GraphConvolution(
    (dropout): Dropout(p=0.7)
  )
  (denseu1): Linear(in_features=15906, out_features=32, bias=True)
  (densev1): Linear(in_features=15906, out_features=32, bias=True)
  (denseu2): Linear(in_features=64, out_features=16, bias=False)
  (densev2): Linear(in_features=64, out_features=16, bias=False)
  (bilin_dec): BilinearMixture(
    (dropout): Dropout(p=0.0)
  )
)
The number of parameters: 3092584


In [10]:
best_epoch = 0
best_loss  = 9999.

## Set optimizer

In [11]:
optimizer = optim.Adam(model.parameters(), 
                       lr = lr, 
                       betas=[beta1, beta2] 
                      )

## Create training and testing function

In [12]:
def reset_grad():
    """Reset the gradient buffers."""
    optimizer.zero_grad()

In [13]:
def train(export):
    global best_loss, best_epoch
    start = time.perf_counter()
    t_loss = []
    t_rmse = []
    v_loss = []
    v_rmse = []
    if start_epoch:
        model.load_state_dict(torch.load(os.path.join(model_path,
                              'model-%d.pkl'%(start_epoch))).state_dict())

    # Training
    for epoch in range(start_epoch, num_epochs):
        model.train()

        train_loss = 0.
        train_rmse = 0.
        for s, u in enumerate(BatchSampler(SequentialSampler(sample(range(num_users), num_users)),
                              batch_size=1024, drop_last=True)): # batch_size = num_users
                              #batch_size=args.batch_size, drop_last=False)):
            u = torch.from_numpy(np.array(u)).long().to(device)


            for t, v in enumerate(BatchSampler(SequentialSampler(sample(range(num_items), num_items)),
                                  batch_size=1024, drop_last=True)): # batch_size =num_items
                                  #batch_size=args.batch_size, drop_last=False)):
                v = torch.from_numpy(np.array(v)).long().to(device)

                if len(torch.nonzero(torch.index_select(torch.index_select(rating_train, 1, u), 2, v))) == 0:
                    continue

                m_hat, loss_ce, loss_rmse = model(u, v, rating_train)

                reset_grad()
                loss_ce.backward()
                optimizer.step()

                train_loss += loss_ce.item()
                train_rmse += loss_rmse.item()

        log = 'epoch: '+str(epoch+1)+' loss_ce: '  +str(train_loss/(s+1)/(t+1)) \
                                    +' loss_rmse: '+str(train_rmse/(s+1)/(t+1))
        print(log)
        t_loss.append(train_loss)
        t_rmse.append(train_rmse)

        if (epoch+1) % val_step == 0:
            # Validation
            model.eval()
            with torch.no_grad():
                u = torch.from_numpy(np.array(range(num_users))).long().to(device)
                v = torch.from_numpy(np.array(range(num_items))).long().to(device)
                m_hat, loss_ce, loss_rmse = model(u, v, rating_val)

            print('[val loss] : '+str(loss_ce.item())+
                  ' [val rmse] : '+str(loss_rmse.item()))
            v_loss.append(loss_ce.item())
            v_rmse.append(loss_rmse.item())
            
            if best_loss > loss_rmse.item():
                best_loss = loss_rmse.item()
                best_epoch= epoch+1
                torch.save(model.state_dict(), os.path.join(model_path, 'model-%d.pkl'%(best_epoch)))
    
    if export == True:
        metrics = {"train_loss": t_loss,
                       "train_rmse": t_rmse,
                       "val_loss": v_loss,
                       "val_rmse": v_rmse     
                  }
        out = open("D:\Python\Thesis\metrics\\gcmc_metrics.pickle","wb")
        pickle.dump(metrics, out)
        out.close()        
    else:
        pass
    
    print("Time elapsed in mins: ", (time.perf_counter() - start)/60)  
    
    return t_loss,t_rmse,v_loss,v_rmse

In [21]:
def test(export):
    t_loss = []
    t_rmse = []
    # Test
    model.load_state_dict(torch.load(os.path.join(model_path,
                          'model-%d.pkl'%(best_epoch))))
    model.eval()
    with torch.no_grad():
        u = torch.from_numpy(np.array(range(num_users))).long().to(device)
        v = torch.from_numpy(np.array(range(num_items))).long().to(device)
        m_hat, loss_ce, loss_rmse = model(u, v, rating_test)
        
    t_loss.append(loss_ce.item())
    t_rmse.append(loss_rmse.item())

    print('[test loss] : '+str(loss_ce.item()) +
          ' [test rmse] : '+str(loss_rmse.item()))
    
    if export == True:
        metrics = {"test_loss": t_loss,
                       "test_rmse": t_rmse,  
                  }
        out = open("D:\Python\Thesis\metrics\\gcmc_test_metrics.pickle","wb")
        pickle.dump(metrics, out)
        out.close()        
    else:
        pass

## Execute training and testing procedure

In [15]:
train_loss, train_rmse, val_loss, val_rmse = train(export = True)

epoch: 1 loss_ce: 0.10563336655364505 loss_rmse: 0.10925683725093092
epoch: 2 loss_ce: 0.01927170343697071 loss_rmse: 0.03861894431923117
epoch: 3 loss_ce: 0.013858786700958652 loss_rmse: 0.031434098325137584
epoch: 4 loss_ce: 0.011711237611182566 loss_rmse: 0.0298767789998757
epoch: 5 loss_ce: 0.010927358978993393 loss_rmse: 0.029117566866001914
[val loss] : 0.014198663644492626 [val rmse] : 0.02953605353832245
epoch: 6 loss_ce: 0.010546791450386601 loss_rmse: 0.029254053752603277
epoch: 7 loss_ce: 0.010647448511528117 loss_rmse: 0.029154642884220396
epoch: 8 loss_ce: 0.010099851925458227 loss_rmse: 0.028839729194130217
epoch: 9 loss_ce: 0.009922067085946245 loss_rmse: 0.027458914056686417
epoch: 10 loss_ce: 0.009597428334278188 loss_rmse: 0.02792585182136723
[val loss] : 0.014952528290450573 [val rmse] : 0.029218221083283424
epoch: 11 loss_ce: 0.009532019383706418 loss_rmse: 0.02751212690158614
epoch: 12 loss_ce: 0.009668753765124296 loss_rmse: 0.02799556913253452
epoch: 13 loss_ce: 

epoch: 97 loss_ce: 0.006553818108971298 loss_rmse: 0.024703431949352046
epoch: 98 loss_ce: 0.006594543991793346 loss_rmse: 0.02477084277364026
epoch: 99 loss_ce: 0.006554275485021728 loss_rmse: 0.024722179905178825
epoch: 100 loss_ce: 0.006686411016354603 loss_rmse: 0.024761985039471517
[val loss] : 0.01471948903053999 [val rmse] : 0.029109638184309006
Time elapsed in mins:  26.797555229483333


In [22]:
test(export = True)

[test loss] : 0.0161051657050848 [test rmse] : 0.027401113882660866
